## Properties / Metadata
The document properties represent the fields that define the data elements you want to retrieve as well as the elements you choose to filter your results.  In addition, properties are at the heart of other Search parameters such as Navigation, Grouping, and Boosting.  There are hundreds of properties available for each View providing critical details needed to fulfill your searching requirements.

In order for you to be successful at search, you will need to understand how to access the available properties, how to determine what values they contain and what they mean.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.DataPlatform.Content, 1.0.0-alpha3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.Data.Analysis version 0.4.0

Installed package Refinitiv.DataPlatform.Content version 1.0.0-alpha3

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.DataPlatform.Content.SearchService;
using Refinitiv.DataPlatform.Delivery.Request;
using Refinitiv.DataPlatform.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Linq;
using System;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<IList<JObject>>((hits, writer) =>
{
    if (hits.Count > 0)
    {
        var fields = new List<String>();
        var rows = new List<ICollection<IHtmlContent>>();
        var data = new Dictionary<string, IHtmlContent>();

        foreach(var hit in hits)
        {
            var cells = new List<IHtmlContent>();
            foreach (var val in hit.Properties())
            {
                if ( !fields.Any(item => item.Equals(val.Name)) )
                    fields.Add(val.Name);
                data[val.Name] = td(val.Value.ToString());
            }
            rows.Add(new List<IHtmlContent>(data.Values));
            data.Keys.ToList().ForEach(x => data[x] = td(""));
        }
        
        var headers = new List<IHtmlContent>();
        headers.AddRange(fields.Select(c => (IHtmlContent)th(c)));

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
var session = CoreFactory.CreateSession(new DesktopSession.Params()
                            .AppKey("Your API Key here")
                            .OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                            .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})")));
session.Open();

17/12/2020 12:25:37 PM:Session is Pending. (State: Pending)
17/12/2020 12:25:37 PM:{
  "Contents": "Desktop Session Successfully Authenticated"
}. (Event: SessionAuthenticationSuccess)
17/12/2020 12:25:37 PM:Session is Opened. (State: Opened)


#### Metadata
MetaData is a convenient service that allows you to list all available properties within a View.  To properly build a metadata list for inspection, you can issue the following:

**Note**: The RDP Libraries for .Net does not currently provide a Content Layer interface for the Search MetaData service.  This will be coming soon.  As such, the code segment below will provide the interface at the Delivery layer.

In [5]:
// Specify the view
var metaDataUrl = "https://api.refinitiv.com/discovery/search/v1/metadata/views/CommodityQuotes";

var eResponse = Endpoint.SendRequest(session, metaDataUrl);
Console.WriteLine(eResponse.Data?.Raw["Properties"]);

{
  "AAACurrencyBondBenchmarkChain": {
    "Type": "String"
  },
  "AACurrencyBondBenchmarkChain": {
    "Type": "String"
  },
  "ABSMBSBondsRIC": {
    "Type": "String"
  },
  "ActivityDate": {
    "Type": "Date",
    "Searchable": true,
    "Sortable": true,
    "Navigable": true
  },
  "ACurrencyBondBenchmarkChain": {
    "Type": "String"
  },
  "AdmissionToTrading": {
    "Type": "Date",
    "Searchable": true,
    "Sortable": true,
    "Navigable": true
  },
  "AliasRIC": {
    "Type": "String",
    "Searchable": true,
    "Exact": true,
    "Symbol": true
  },
  "AllATMOptionsContinuationChainsRIC": {
    "Type": "String"
  },
  "AllStrikesOptionsChainRIC": {
    "Type": "String"
  },
  "AlternateISIN": {
    "Type": "String",
    "Searchable": true,
    "Symbol": true
  },
  "AnalyticRIC": {
    "Type": "String",
    "Searchable": true,
    "Symbol": true
  },
  "ApplicableInventory": {
    "Type": "String"
  },
  "AssetCategory": {
    "Type": "String",
    "Searchable": true,


#### Debugging (*_debugall*)
While MetaData will help list all properties available within a View, all properties will not be available for use.  That is, depending on the type of search you perform, will determine which properties will be populated - not all will be.  This is where users can use a technique to list all available properties that are populated, for the specific search requested.

In [6]:
// Note: It is critical you select one document top=1 because the amount of data return is very large
var response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                       .Top(1)   
                                                       .Select("_debugall")
                                                       .GetData();
Console.WriteLine(response.Data?.Raw);

{
  "Total": 41648,
  "Hits": [
    {
      "meta": {
        "_id": "37036",
        "_index": "eikonsearch_organisation_1.132.0",
        "_score": 2838761.0,
        "_timestamp": 1608224590916,
        "_timestamp_human": "2020-12-17T17:03:10.916Z",
        "_version": 1608222599000
      },
      "raw_source": {
        "RCSOrganisationTypeGenealogy": "OT:1",
        "SPsRatingOutlookX1XSPsRatingOutlookRank": "SPI/NegativeX1X-1|SPI/NegativeX1X-1",
        "RCSTRBCGenealogy": "B:161\\B:171\\B:172\\B:173",
        "IssuerID": "0x000019000050e3aa",
        "SicSchemeCode": "8742",
        "OfficersExist": true,
        "IssuerBondsExist": true,
        "CertificatesOfDepositCount": 0,
        "MktCapCompanyUsd": 111872220913,
        "TotalAssetsUsd": 152186000000,
        "GEMPermID": "4295904307",
        "SubjectBoost": "firm",
        "EEDBID": "14129",
        "SigDevExist": true,
        "SDCID": "3104001",
        "TotalAssets": 152186000000,
        "EstimatesExist": true,
  

#### Understanding values using Navigators
Using the output from a debugging session, there are a number of common properties that are very useful when filtering your search.  For example, you may want to filter your results based on a specific category of data.  Using navigators will help us understand these categories.

Looking at our debug output above, one useful property: **SearchAllCategoryv3** will helps us understand all available categories of documents the query term 'IBM' was found.  To list these categories, use the following:

In [7]:
response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                   .Top(0)
                                                   .Navigators("SearchAllCategoryv3")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["SearchAllCategoryv3"]["Buckets"]);

[
  {
    "Label": "Other Quote",
    "Count": 25293
  },
  {
    "Label": "Warrants",
    "Count": 6785
  },
  {
    "Label": "Bond Pricing",
    "Count": 2028
  },
  {
    "Label": "Bonds",
    "Count": 1414
  },
  {
    "Label": "Commercial Papers",
    "Count": 1282
  },
  {
    "Label": "Options",
    "Count": 1050
  },
  {
    "Label": "Companies/Issuers",
    "Count": 774
  },
  {
    "Label": "Futures",
    "Count": 749
  },
  {
    "Label": "Mergers & Acquisitions",
    "Count": 713
  },
  {
    "Label": "Equities",
    "Count": 494
  },
  {
    "Label": "Credit Default Swaps",
    "Count": 463
  },
  {
    "Label": "Analyst",
    "Count": 237
  },
  {
    "Label": "Officers",
    "Count": 176
  },
  {
    "Label": "FX & Money",
    "Count": 66
  },
  {
    "Label": "Fixed Income Curves",
    "Count": 48
  },
  {
    "Label": "Other Instrument",
    "Count": 23
  },
  {
    "Label": "Mortgage Backed Securities",
    "Count": 12
  },
  {
    "Label": "Funds",
    "Count": 10
  

Using the above listing, we can narrow our result set based on these categories using a Filter.

#### Special Properties
The Search service includes a number of special properties that enhance the ability to find meaningful results. In addition to the special property *_debugall* used within the **select** parameter as outlined above, the following properties are available: 

In [8]:
// The 'code' specification will search across all codes, such as RICs, tickers, SEDOLs etc.  
// Primarily used within a filter but can be used as a selectable property.
response = Search.Definition(Search.View.SearchAll).Filter("code eq 'IBM'")
                                                   .GetData();
response.Data.Hits

BusinessEntity DocumentTitle PI PermID RIC ORGANISATION International Business Machines Corp, Public Company 37036 QUOTExEQUITY International Business Machines Corp, Ordinary Share, NYSE Consolidated 1097326 55839165994 IBM QUOTExEQUITY International Business Machines Corp, Ordinary Share, MiFID Composite Cross Market Service 79345498 21475739068 IBMEUR.xbo QUOTExEQUITY International Business Machines Corp, Depository Receipt, Bolsa de Comercio de Buenos Aires 1006058 55838283585 IBM.BA QUOTExEQUITY International Business Machines Corp, Ordinary Share, New York Stock Exchange 1090370 55838323096 IBM.N QUOTExEQUITY International Business Machines Corp, Ordinary Share, NASDAQ INTERMARKET 765744 55835333953 IBM.TH QUOTExEQUITY International Business Machines Corp, Ordinary Share, Cboe BZX Exchange 53703996 55845629228 IBM.Z QUOTExEQUITY International Business Machines Corp, Ordinary Share, NYSE Arca 753753 55837129318 IBM.P QUOTExEQUITY International Business Machines Corp, Ordinary Share, Cboe EDGX Exchange 73533774 21475059739 IBM.DG QUOTExEQUITY International Business Machines Corp, Ordinary Share, Frankfurt Stock Exchange 722691 55836444333 IBM.F

In [9]:
// The 'name' specification will search across all names, such as common name, legal name, issuer name, etc. 
// Primarily used within a filter but can be used as a selectable property.
response = Search.Definition(Search.View.SearchAll).Filter("name eq 'IBM'")
                                                   .GetData();
response.Data.Hits

BusinessEntity DocumentTitle PI PermID RIC ORGANISATION International Business Machines Corp, Public Company 37036 QUOTExEQUITY International Business Machines Corp, Ordinary Share, NYSE Consolidated 1097326 55839165994 IBM ORGANISATION IBM Credit LLC, Private Company 99244 ORGS IBM Retirement Fund Trust, Private Company 4297694029 ORGANISATION Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company 18062670 ORGANISATION SG Stuttgart Vaihingen BM-Campus 1 UG haftungsbeschraenkt, Private Company 27968389 QUOTExEQUITY Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex 48924732 21481052421 0#IBMF: QUOTExEQUITY Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam 259118763 21612423771 0#IBMDF: QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex 49450681 21481052892 IBMFc1 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex 50092347 21481053949 IBMFc2

In [10]:
// The special character underscore (_) is a convenient token that can be used to select the default Properties.  
// This will allow users to quickly select the default values plus any other properties when testing out your search.
response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                   .Top(1000)
                                                   .Select("_, SearchAllCategoryv3")
                                                   .GetData();
response.Data.Hits

BusinessEntity DocumentTitle PI SearchAllCategoryv3 PermID RIC ORGANISATION International Business Machines Corp, Public Company 37036 Companies/Issuers QUOTExEQUITY International Business Machines Corp, Ordinary Share, NYSE Consolidated 1097326 Equities 55839165994 IBM ORGANISATION IBM Credit LLC, Private Company 99244 Companies/Issuers ORGS IBM Retirement Fund Trust, Private Company 4297694029 Companies/Issuers ORGANISATION Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company 18062670 Companies/Issuers ORGANISATION SG Stuttgart Vaihingen BM-Campus 1 UG haftungsbeschraenkt, Private Company 27968389 Companies/Issuers QUOTExEQUITY Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex 48924732 Futures 21481052421 0#IBMF: QUOTExEQUITY Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam 259118763 Futures 21612423771 0#IBMDF: QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex 49450681 Futures 21481052892 IBMFc1 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex 50092347 Futures 21481053949 IBMFc2 QUOTExEQUITY Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam 260213021 Futures 21613372305 IBMDFc1 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex 50092348 Futures 21481053950 IBMFc3 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 4, Equity Future, USD, Eurex 50092349 Futures 21481053951 IBMFc4 QUOTExEQUITY Euronext Amsterdam IBM Single Stock Dividend Future Continuation 2, Equity Future, USD, Euronext Amsterdam 260213022 Futures 21613372306 IBMDFc2 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 5, Equity Future, USD, Eurex 50092350 Futures 21481053952 IBMFc5 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 6, Equity Future, USD, Eurex 50092351 Futures 21481053953 IBMFc6 QUOTExEQUITY Euronext Amsterdam IBM Single Stock Dividend Future Continuation 3, Equity Future, USD, Euronext Amsterdam 260213023 Futures 21613372307 IBMDFc3 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 7, Equity Future, USD, Eurex 50092352 Futures 21481053954 IBMFc7 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 8, Equity Future, USD, Eurex 50092353 Futures 21481053955 IBMFc8 QUOTExEQUITY Euronext Amsterdam IBM Single Stock Dividend Future Continuation 4, Equity Future, USD, Euronext Amsterdam 260213024 Futures 21613372308 IBMDFc4 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 9, Equity Future, USD, Eurex 50092354 Futures 21481053956 IBMFc9 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 10, Equity Future, USD, Eurex 50092355 Futures 21481053957 IBMFc10 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 11, Equity Future, USD, Eurex 50092356 Futures 21481053958 IBMFc11 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 12, Equity Future, USD, Eurex 50092357 Futures 21481053959 IBMFc12 QUOTExEQUITY Euronext Amsterdam IBM Single Stock Dividend Future Continuation 5, Equity Future, USD, Euronext Amsterdam 260213025 Futures 21613372309 IBMDFc5 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 13, Equity Future, USD, Eurex 50092358 Futures 21481053960 IBMFc13 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 14, Equity Future, USD, Eurex 50092359 Futures 21481053961 IBMFc14 QUOTExEQUITY Eurex International Business Machines Equity Future Continuation 15, Equity Future, USD, Eurex 50092360 Futures 21481053962 IBMFc15 INSTRUMENTxINDEX CBOE IBM VIX Index, Equity Index 80032992 Indices .VXIBM QUOTExEQUITY International Busine

#### Common Properties
As you become more familiar with search, you will soon recognize there will be a number of very useful properties that will act as your "go-tos" when filtering or selecting fields.

In [11]:
// DocumentTitle
// As you've seen in some of the above output, this property is on by default when you want to display your search results.  
// The DocumentTitle property is a quick way to describe the content retrieved for each matching document.  
// It is made up of the following child properties: 
//    'DTSubjectName', 'DTSimpleType', 'DTSource', and 'DTCharacteristics'.
var select = "DocumentTitle, DTSubjectName, DTSimpleType, DTSource, DTCharacteristics";

response = Search.Definition(Search.View.SearchAll).Select(select)
                                                   .Query("IBM")
                                                   .Top(100)
                                                   .GetData();
response.Data.Hits

DocumentTitle DTSubjectName DTSimpleType DTSource DTCharacteristics International Business Machines Corp, Public Company International Business Machines Corp Public Company International Business Machines Corp, Ordinary Share, NYSE Consolidated International Business Machines Corp Ordinary Share NYSE Consolidated IBM Credit LLC, Private Company IBM Credit LLC Private Company IBM Retirement Fund Trust, Private Company IBM Retirement Fund Trust Private Company Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4, Private Company Tiers Corporate Bond Backed Certificates Trust Series Ibm 1997 4 Private Company SG Stuttgart Vaihingen BM-Campus 1 UG haftungsbeschraenkt, Private Company SG Stuttgart Vaihingen BM-Campus 1 UG haftungsbeschraenkt Private Company Eurex International Business Machines Equity Future Chain Contract , Equity Future, USD, Eurex Eurex International Business Machines Equity Future Chain Contract Equity Future Eurex USD Euronext Amsterdam IBM Dividend Future Chain Contracts, Equity Future, USD, Euronext Amsterdam Euronext Amsterdam IBM Dividend Future Chain Contracts Equity Future Euronext Amsterdam USD Eurex International Business Machines Equity Future Continuation 1, Equity Future, USD, Eurex Eurex International Business Machines Equity Future Continuation 1 Equity Future Eurex USD Eurex International Business Machines Equity Future Continuation 2, Equity Future, USD, Eurex Eurex International Business Machines Equity Future Continuation 2 Equity Future Eurex USD Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1, Equity Future, USD, Euronext Amsterdam Euronext Amsterdam IBM Single Stock Dividend Future Continuation 1 Equity Future Euronext Amsterdam USD Eurex International Business Machines Equity Future Continuation 3, Equity Future, USD, Eurex Eurex International Business Machines Equity Future Continuation 3 Equity Future Eurex USD Eurex International Business Machines Equity Future Continuation 4, Equity Future, USD, Eurex Eurex International Business Machines Equity Future Continuation 4 Equity Future Eurex USD Euronext Amsterdam IBM Single Stock Dividend Future Continuation 2, Equity Future, USD, Euronext Amsterdam Euronext Amsterdam IBM Single Stock Dividend Future Continuation 2 Equity Future Euronext Amsterdam USD Eurex International Business Machines Equity Future Continuation 5, Equity Future, USD, Eurex Eurex International Business Machines Equity Future Continuation 5 Equity Future Eurex USD Eurex International Business Machines Equity Future Continuation 6, Equity Future, USD, Eurex Eurex International Business Machines Equity Future Continuation 6 Equity Future Eurex USD Euronext Amsterdam IBM Single Stock Dividend Future Continuation 3, Equity Future, USD, Euronext Amsterdam Euronext Amsterdam IBM Single Stock Dividend Future Continuation 3 Equity Future Euronext Amsterdam USD Eurex International Business Machines Equity Future Continuation 7, Equity Future, USD, Eurex Eurex International Business Machines Equity Future Continuation 7 Equity Future Eurex USD Eurex International Business Machines Equity Future Continuation 8, Equity Future, USD, Eurex Eurex International Business Machines Equity Future Continuation 8 Equity Future Eurex USD Euronext Amsterdam IBM Single Stock Dividend Future Continuation 4, Equity Future, USD, Euronext Amsterdam Euronext Amsterdam IBM Single Stock Dividend Future Continuation 4 Equity Future Euronext Amsterdam USD Eurex International Business Machines Equity Future Continuation 9, Equity Future, USD, Eurex Eurex International Business Machines Equity Future Continuation 9 Equity Future Eurex USD Eurex International Business Machines Equity Future Continuation 10, Equity Future, USD, Eurex Eurex International Business Machines Equity Future Continuation 10 Equity Future Eurex USD Eurex International Business Machines Equity Future Continuation 11, Equity Future, USD, Eurex Eurex International Business Machines Equity Future Continuat

In [12]:
// SearchAllCategoryv3
// When trying to narrow down your search to specific categories, this property can be extremely useful.  
// When you apply a Navigator on this property, it will return a list of ~40 categories, depending on the view. 
// Categories include asset-related instruments such as:
//     'Equities', 'Futures' , 'Options', 'Bonds', 'Commodities' 
//     but can also include things like: 'People', 'Analyst', 'Officers'.
response = Search.Definition(Search.View.SearchAll).Query("IBM")
                                                   .Filter("SearchAllCategoryv3 eq 'People'")
                                                   .GetData();
response.Data.Hits

PERSON,John Denham - IsUnknown,34414661717,34414661717
PERSON,Charlie Maxfield - IsUnknown,34414669683,34414669683
PERSON,Claudia Barclay - IsUnknown,34414721056,34414721056
PERSON,Robert Ploder - IsUnknown,34414738667,34414738667
PERSON,Tom Cadigan - IsUnknown,34414743082,34414743082
PERSON,R. L. Jay Vivian - IsUnknown,34414808510,34414808510
PERSON,William E. Baker - IsUnknown,34415092341,34415092341


In [13]:
// CodeResolutionBase
// Represents the most significant codes defined within the document.  For example, when searching for a symbol such 
// as ISIN, SEDOL, RIC, etc, filtering against this property will significantly reduce the matches and only bring 
// back the most relevant results.
response = Search.Definition(Search.View.SearchAll).Filter("CodeResolutionBase eq 'US046353AQ14'")
                                                   .Select("DocumentTitle, ISIN, SEDOL, RIC")
                                                   .GetData();
response.Data.Hits

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,BF4SNM7,046353AQ1=


In [14]:
// Conversely, if we were to choose the more generic term 'code', would result in a wider range of matches.
response = Search.Definition(Search.View.SearchAll).Filter("code eq 'US046353AQ14'")
                                                   .Select("DocumentTitle, ISIN, SEDOL, RIC")
                                                   .GetData();
response.Data.Hits

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",US046353AQ14,BF4SNM7,046353AQ1=
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS L2",,BF4SNM7,046353AQ1=2M
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, CREDIT SUISSE TRADEWEB PRICE",,BF4SNM7,US046353AQ14=CSTW
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS COMPOSITE PLUS",,BF4SNM7,US046353AQ14=MKCP
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TDS TRADEWEB PRICE",,BF4SNM7,US046353AQ14=TDTW
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TRADEWEB",,BF4SNM7,US046353AQ14=TE
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TRADEWEB",,BF4SNM7,046353AQ1=TWEB
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, GFI GROUP",,BF4SNM7,US046353AQ14=GFGL
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, MARKETAXESS AXESSALL",,BF4SNM7,US046353AQ14=MKAX
"AstraZeneca PLC, Bond Quote, TRACE Eligible AZN 2.375 12-Jun-2022, TWEB EUCR MARKS",,BF4SNM7,US046353AQ14=TEMK


#### Common Properties - RCS (Reuters Classification Scheme)
Throughout many documents, you will notice a number of RCS fields that are popular when selecting and filtering.

*Note* - I'll use navigators to demonstrate the values of these common properties.

In [15]:
// RCSAssetCategoryLeaf
// Specific to assets, this property will categorize the results into a readable identifier such as: 'Bond', 
// 'Commodity Future', 'Commercial Paper', etc.  Where the property SearchAllCategoryv3 provides general categories, 
// this property will be more specific to assets.  Pull out the top 10.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSAssetCategoryLeaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSAssetCategoryLeaf"]["Buckets"]);

[
  {
    "Label": "Barrier Warrant",
    "Count": 15802746
  },
  {
    "Label": "Bond",
    "Count": 13622037
  },
  {
    "Label": "Commodity Future",
    "Count": 4535314
  },
  {
    "Label": "Traditional Warrant",
    "Count": 3828011
  },
  {
    "Label": "Equity Cash Option",
    "Count": 3266707
  },
  {
    "Label": "Commodity Future Option",
    "Count": 3073029
  },
  {
    "Label": "Equity Future",
    "Count": 2667403
  },
  {
    "Label": "Commodity Spread",
    "Count": 2663936
  },
  {
    "Label": "Certificate of Deposit",
    "Count": 2628904
  },
  {
    "Label": "Mortgage-Backed Security",
    "Count": 2616506
  },
  {
    "Label": "Interest Rate Future Spread",
    "Count": 2608504
  },
  {
    "Label": "Commercial Paper",
    "Count": 1994825
  },
  {
    "Label": "Commodity Spot",
    "Count": 1794084
  },
  {
    "Label": "Ordinary Share",
    "Count": 1620066
  },
  {
    "Label": "Buffered Risk / Bonus Certificate",
    "Count": 1564890
  },
  {
    "Label": 

In [16]:
// RCSCountryLeaf
// RCSIssuerCountryLeaf
// RCSExchangeCountryLeaf
// Specific to some assets, will allow you to narrow your search to focus on a specific country.  Pull out the top 20.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSCountryLeaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSCountryLeaf"]["Buckets"]);

[
  {
    "Label": "United States",
    "Count": 11850499
  },
  {
    "Label": "Eurobond Market",
    "Count": 3613299
  },
  {
    "Label": "Colombia",
    "Count": 1266336
  },
  {
    "Label": "China (Mainland)",
    "Count": 1227295
  },
  {
    "Label": "South Korea",
    "Count": 533785
  },
  {
    "Label": "Germany",
    "Count": 469964
  },
  {
    "Label": "Canada",
    "Count": 374106
  },
  {
    "Label": "France",
    "Count": 221276
  },
  {
    "Label": "India",
    "Count": 186144
  },
  {
    "Label": "Japan",
    "Count": 156881
  },
  {
    "Label": "Chile",
    "Count": 152045
  },
  {
    "Label": "Italy",
    "Count": 135890
  },
  {
    "Label": "Australia",
    "Count": 86807
  },
  {
    "Label": "Spain",
    "Count": 58654
  },
  {
    "Label": "Switzerland",
    "Count": 58418
  },
  {
    "Label": "Russia",
    "Count": 56574
  },
  {
    "Label": "Austria",
    "Count": 55442
  },
  {
    "Label": "Norway",
    "Count": 52863
  },
  {
    "Label": "United 

In [17]:
// RCSCurrencyLeaf
// Specific to some assets, will allow you to narrow your search to focus on a specific currency.  Just pull out the top 20.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSCurrencyLeaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSCurrencyLeaf"]["Buckets"]);

[
  {
    "Label": "US Dollar",
    "Count": 31979670
  },
  {
    "Label": "Euro",
    "Count": 29909281
  },
  {
    "Label": "Chinese Yuan",
    "Count": 2062460
  },
  {
    "Label": "US Cent",
    "Count": 1645130
  },
  {
    "Label": "Colombian Peso",
    "Count": 1439387
  },
  {
    "Label": "Japanese Yen",
    "Count": 996648
  },
  {
    "Label": "Canadian Dollar",
    "Count": 856052
  },
  {
    "Label": "Hong Kong Dollar",
    "Count": 852975
  },
  {
    "Label": "Swiss Franc",
    "Count": 782729
  },
  {
    "Label": "Swedish Krona",
    "Count": 745040
  },
  {
    "Label": "British Pound",
    "Count": 725822
  },
  {
    "Label": "Korean (South) Won",
    "Count": 701974
  },
  {
    "Label": "Indian Rupee",
    "Count": 700280
  },
  {
    "Label": "Australian Dollar",
    "Count": 445607
  },
  {
    "Label": "British Penny",
    "Count": 440872
  },
  {
    "Label": "Hungarian Forint",
    "Count": 349327
  },
  {
    "Label": "Norwegian Krone",
    "Count": 3255

In [18]:
// RCSTRBC2012Leaf
// Represents the industry sector.  For example, if I query for Apple Computers, I will get many different 
// industry sectors such as: 'Computer Hardware (NEC)', 'Laptop & Desktop Computers', 'Phones & Smart Phones', etc.
response = Search.Definition(Search.View.SearchAll).Top(0)
                                                   .Navigators("RCSTRBC2012Leaf(buckets:20)")
                                                   .GetData();
Console.WriteLine(response.Data.Navigators["RCSTRBC2012Leaf"]["Buckets"]);

[
  {
    "Label": "Banks (NEC)",
    "Count": 3675141
  },
  {
    "Label": "Corporate Financial Services (NEC)",
    "Count": 2810241
  },
  {
    "Label": "Corporate Banks",
    "Count": 1640432
  },
  {
    "Label": "Retail & Mortgage Banks",
    "Count": 1273154
  },
  {
    "Label": "Public Finance Activities",
    "Count": 898432
  },
  {
    "Label": "Investment Banking & Brokerage Services (NEC)",
    "Count": 594579
  },
  {
    "Label": "Consumer Lending (NEC)",
    "Count": 378283
  },
  {
    "Label": "Investment Management & Fund Operators (NEC)",
    "Count": 353936
  },
  {
    "Label": "Electric Utilities (NEC)",
    "Count": 283590
  },
  {
    "Label": "Investment Holding Companies (NEC)",
    "Count": 276633
  },
  {
    "Label": "Diversified Investment Services",
    "Count": 262283
  },
  {
    "Label": "Wealth Management",
    "Count": 248669
  },
  {
    "Label": "Construction & Engineering (NEC)",
    "Count": 240612
  },
  {
    "Label": "Real Estate Rental, D

In [20]:
// RCSTRBC2012Leaf
// RCSTRBC2012Genealogy
// RCSTRBC2012Name
// Provide a breakdown of the hierarchy defined for many of these RCS attributes.  You can see the mapping between the 
// hierarchy defined with the Genealogy property and Name property.  
// For example, the code 'B:262' maps to the name 'Banking & Investment Services'.
response = Search.Definition(Search.View.SearchAll).Filter("RCSTRBC2012Leaf eq 'Banks'")
                                                   .Top(0)
                                                   .Navigators("RCSTRBC2012Leaf, RCSTRBC2012Genealogy, RCSTRBC2012Name")
                                                   .GetData();
Console.WriteLine(response.Data.Raw);

{
  "Total": 6650224,
  "Hits": [],
  "Navigators": {
    "RCSTRBC2012Genealogy": {
      "Buckets": [
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1615",
          "Count": 3647319
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1616",
          "Count": 1624175
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1617",
          "Count": 1264379
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1619",
          "Count": 35427
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:131\\B:1629",
          "Count": 20054
        },
        {
          "Label": "B:261\\B:262\\B:263\\B:207\\B:1638",
          "Count": 3859
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1618",
          "Count": 495
        },
        {
          "Label": "B:261\\B:262\\B:127\\B:128\\B:1620",
          "Count": 364
        }
      ]
    },
    "RCSTRBC2012Leaf": {
      "Buckets": [
        {
    

The above output includes 3 separate sections: RCSTRBC2012Genealogy, RCSTRBC2012Leaf and RCSTRBC2012Name.  For each you can see a breakdown ot the buckets.